In [2]:
from ipynb.fs.full.Background_Functions import *
from ipynb.fs.full._03_Cells_with_No_Boundaries import *

# Fixed Displacement BCs

## $a$ terms

For the boundary cells, the only term that will be affected is the term assosciated with the boundary face, i.e. the $N$ term for the top boundary, the $S$ term for the bottom boundary,  the $E$ term for the right boundary, the $W$ term for the left boundary

Below shows an example of the top boundary and how this discretization effects the x and y momentum equations:

<img src="./Paper_Images/boundaryCellDisplacement.png" alt="Drawing" style="width: 600px;"/> 


### Top Boundary:

### x-equation:

$$
\frac{\rho}{\Delta t^2} \left( u_P^n V^n \right) 
-
\mu \left( \dfrac{ u_N - u_P }{dy\color{red}{/2}} \right)
 |S_N|
+
\mu \left( \dfrac{ u_P - u_S }{dy} \right)
  |S_S|
- 
(2\mu + \lambda)
\left(\dfrac{ u_E - u_P}{dx} \right)
  |S_E|
+ 
(2\mu + \lambda)
\left(\dfrac{ u_P - u_W}{dx}\right) 
  |S_W|
= 
RHS
$$

$$
u_P \left[
\frac{\rho}{\Delta t^2} V^n
+ 
\color{red} 2\dfrac{\mu |S_N|}{dy} 
+
\dfrac{\mu |S_S|}{dy} 
+
\dfrac{(2\mu + \lambda) |S_E|}{dx}
+
\dfrac{(2\mu + \lambda) |S_W|}{dx}
\right]
-
\color{red} 2 u_N \dfrac{\mu |S_N|}{dy}
-
u_S \dfrac{\mu |S_S|}{dy}
- 
u_E \dfrac{(2\mu + \lambda) |S_E|}{dx} 
- 
u_W \dfrac{(2\mu + \lambda) |S_W|}{dx} 
= 
RHS
$$

<!-- ### y-equation:

$$
\frac{\rho}{\Delta t^2} \left(   v_p^n V^n \right)
-
(2\mu + \lambda)
\left(\dfrac{\delta v}{dy}\right) 
 _N |S_N|

+
(2\mu + \lambda)
\left(\dfrac{\delta v}{dy} \right)
_S |S_S|
- 
\mu
\left( \dfrac{\delta v}{dx}\right)
 _E |S_E|
+ 
\mu
\left( \dfrac{\delta v}{dx}\right)
 _W |S_W|
= 
RHS
$$ -->


### y-equation:

$$
\frac{\rho}{\Delta t^2} \left(   v_p^n V^n \right)
-
(2\mu + \lambda)
\left(\dfrac{v_N - v_P}{dy\color{red}{/2}}\right) 
  |S_N|
+
(2\mu + \lambda)
\left(\dfrac{v_P - v_S}{dy} \right)
 |S_S|
- 
\mu
\left( \dfrac{v_E - v_P}{dx}\right)
  |S_E|
+ 
\mu
\left( \dfrac{v_P - v_W}{dx}\right)
  |S_W|
= 
RHS
$$

$$
v_P \left[
\frac{\rho}{\Delta t^2} V^n
+ 
\color{red} 2 \dfrac{(2\mu + \lambda)  |S_N|}{dy} 
+
\dfrac{(2\mu + \lambda)  |S_S|}{dy} 
+
\dfrac{\mu |S_E|}{dx}
+
\dfrac{\mu |S_W|}{dx}
\right]
-
\color{red} 2 v_N \dfrac{(2\mu + \lambda) |S_N|}{dy}
-
v_S \dfrac{(2\mu + \lambda) |S_S|}{dy}
- 
v_E \dfrac{\mu |S_E|}{dx} 
- 
v_W \dfrac{\mu |S_W|}{dx} 
= 
RHS
$$

Essentially, the terms for the A matrix are the same for the non-boundary cells except the terms associated with the boundary face are doubled

In [3]:
xy = "x"
edges = ["b"]

a_N = A(xy).a_N
a_S = A(xy).a_S
a_E = A(xy).a_E
a_W = A(xy).a_W

# Double a terms if on the boundary
for edge in edges:
    if edge == "b": a_S = A(xy).a_S*2
    if edge == "t": a_N = A(xy).a_N*2
    if edge == "l": a_W = A(xy).a_W*2
    if edge == "r": a_E = A(xy).a_E*2

if transient:
    a_P = (rho*dx*dy/(dt**2)) + a_N + a_S + a_E + a_W
else:
    a_P = a_N + a_S + a_E + a_W

### $b$ term

For the $b$ term, or RHS of the momentum equation, nothing changes from the cells without a boundary exacept the approximation of the corner displacement values. 

Instead of using the average of the surrounding cell centre centres, we use the average of the displacements of the face centres either side of the corner on the boundary:

<img src="./Paper_Images/displacementCellDisplacementEdgeCorners.png" alt="Drawing" style="width: 600px;"/> 

In [5]:
def corner(edges, corner_placement, uv, U_array, k):
    if uv == "u":
        uv_i = 0
    elif uv == "v":
        uv_i = 1

    # the displacement function returns the u or v values (uv_i) for a particular cell (k) within a displacement field (U_array)
    # disp below is the u or v value at cell k 

    # original corner expression
    if corner_placement == "NE":
        corner =  (1/4)*(disp.P + disp.NE + disp.N + disp.E)
    if corner_placement == "SE":
        corner =  (1/4)*(disp.P + disp.SE + disp.S + disp.E)
    if corner_placement == "SW":
        corner =  (1/4)*(disp.P + disp.SW + disp.S + disp.W)
    if corner_placement == "NW":
        corner =  (1/4)*(disp.P + disp.NW + disp.N + disp.W)

    # update corner value if on the boundary
    for edge in edges:
        if (edge == "b") & (corner_placement == "SE"):
            corner =  (1/2)*(disp.SE + disp.S)
        if (edge == "b") & (corner_placement == "SW"):
            corner =  (1/2)*(disp.SW + disp.S)

        if (edge == "t") & (corner_placement == "NE"):
            corner =  (1/2)*(disp.NE + disp.N)
        if (edge == "t") & (corner_placement == "NW"):
            corner =  (1/2)*(disp.NW + disp.N)

        if (edge == "l") & (corner_placement == "NW"):
            corner =  (1/2)*(disp.NW + disp.W)
        if (edge == "l") & (corner_placement == "SW"):
            corner =  (1/2)*(disp.SW + disp.W)

        if (edge == "r") & (corner_placement == "NE"):
            corner =  (1/2)*(disp.NE + disp.E)
        if (edge == "r") & (corner_placement == "SE"):
            corner =  (1/2)*(disp.SE + disp.E)

    return corner

Creating a class containing all this:

In [6]:
class boundaryCellDisplacement(A):

    def __init__(self, edges, xy):

        self.a_N = A(xy).a_N
        self.a_S = A(xy).a_S
        self.a_E = A(xy).a_E
        self.a_W = A(xy).a_W
        
        # Double a terms if on the boundary
        for edge in edges:
            if edge == "b": self.a_S = A(xy).a_S*2
            if edge == "t": self.a_N = A(xy).a_N*2
            if edge == "l": self.a_W = A(xy).a_W*2
            if edge == "r": self.a_E = A(xy).a_E*2

        if transient:
            self.a_P = (rho*dx*dy/(dt**2)) + self.a_N + self.a_S + self.a_E + self.a_W
        else:
            self.a_P = self.a_N + self.a_S + self.a_E + self.a_W

    def b_temp(U_old, U_old_old, k, xy):      
        return A.b_temp(U_old, U_old_old, k, xy)
    
    def b_diff(edges, k, xy, U_previous):

        if xy == "x":
            uv = "v"
        if xy == "y":
            uv = "u"
            
        N_term =(
                    S_N*A.coef(xy, "N", uv)*(
                    (boundaryCellDisplacement.corner(edges, "NE", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "NW", uv, U_previous, k))
                    /dx)
                )
        S_term =(
                    S_S*A.coef(xy, "S", uv)*(
                        (boundaryCellDisplacement.corner(edges, "SE", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "SW", uv, U_previous, k))
                        /dx)
                ) 
        E_term =(
                    S_E*A.coef(xy, "E", uv)*(
                        (boundaryCellDisplacement.corner(edges, "NE", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "SE", uv, U_previous, k))
                        /dy)
                ) 
        W_term =(
                    S_W*A.coef(xy, "W", uv)*(
                        (boundaryCellDisplacement.corner(edges, "NW", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "SW", uv, U_previous, k))
                        /dy)
                ) 

        b_diffusion = (N_term + S_term + E_term + W_term)

        return b_diffusion
    
    def corner(edges, corner_placement, uv, U_previous, k):

        if uv == "u":
            uv_i = 0
        elif uv == "v":
            uv_i = 1

        disp = displacement(k, U_previous, uv_i)

        if corner_placement == "NE":
            corner =  (1/4)*(disp.P + disp.NE + disp.N + disp.E)
        if corner_placement == "SE":
            corner =  (1/4)*(disp.P + disp.SE + disp.S + disp.E)
        if corner_placement == "SW":
            corner =  (1/4)*(disp.P + disp.SW + disp.S + disp.W)
        if corner_placement == "NW":
            corner =  (1/4)*(disp.P + disp.NW + disp.N + disp.W)

        for edge in edges:
            if (edge == "b") & (corner_placement == "SE"):
                corner =  (1/2)*(disp.SE + disp.S)
            if (edge == "b") & (corner_placement == "SW"):
                corner =  (1/2)*(disp.SW + disp.S)

            if (edge == "t") & (corner_placement == "NE"):
                corner =  (1/2)*(disp.NE + disp.N)
            if (edge == "t") & (corner_placement == "NW"):
                corner =  (1/2)*(disp.NW + disp.N)

            if (edge == "l") & (corner_placement == "NW"):
                corner =  (1/2)*(disp.NW + disp.W)
            if (edge == "l") & (corner_placement == "SW"):
                corner =  (1/2)*(disp.SW + disp.W)

            if (edge == "r") & (corner_placement == "NE"):
                corner =  (1/2)*(disp.NE + disp.E)
            if (edge == "r") & (corner_placement == "SE"):
                corner =  (1/2)*(disp.SE + disp.E)

        # This part if for the corner point displacements

        if len(edges) > 1:

            if (edges[0] == "b") & (edges[1] == "l") & (corner_placement == "SW"):
                corner = disp.SW 
            if (edges[0] == "b") & (edges[1] == "r") & (corner_placement == "SE"):
                corner = disp.SE
            if (edges[0] == "t") & (edges[1] == "l") & (corner_placement == "NW"):
                corner = disp.NW
            if (edges[0] == "t") & (edges[1] == "r") & (corner_placement == "NE"):
                corner = disp.NE

        
        return corner

# boundaryCellDisplacement.b_diff(["b", "l"], 14, "x", U_previous)

#### Test Case setup: 

The test case used will be a cantilever beam. One end of the beam will be fixed, i.e. the boundary condition is fixed displacement of 0 m. The top and bottom edges will have a traction BC of 0, the loaded end has a fixed traction BC in the negative y-direction. 

In [7]:
# Cantilever Setup 

tr_right_x = 0    #u boundary condition at the right boundary
tr_right_y = - 1e6   #v boundary condition at the right boundary

tr_top_x = 0    #u boundary condition at the top boundary
tr_top_y = 0       #v boundary condition at the top boundary

tr_bottom_x = 0    #u boundary condition at the bottom boundary 
tr_bottom_y = 0  #v boundary condition at the bottom boundary

u_left = 0
v_left = 0

In [8]:
# Set Boundary Conditions:
class BC_settings:

    left = "fixed_displacement"
    right = "traction"
    top = "traction"
    bottom = "traction"

    def __init__(self, edge):

        if edge == "l":
            if BC_settings.left == "traction":
                self.traction = True
                self.fixed_displacement = False
            elif BC_settings.left == "fixed_displacement":
                self.fixed_displacement = True
                self.traction = False

        if edge == "r":
            if BC_settings.right == "traction":
                self.traction = True
                self.fixed_displacement = False
            elif BC_settings.right == "fixed_displacement":
                self.fixed_displacement = True
                self.traction = False

        if edge == "t":
            if BC_settings.top == "traction":
                self.traction = True
                self.fixed_displacement = False
            elif BC_settings.top == "fixed_displacement":
                self.fixed_displacement = True
                self.traction = False

        if edge == "b":
            if BC_settings.bottom == "traction":
                self.traction = True
                self.fixed_displacement = False
            elif BC_settings.bottom == "fixed_displacement":
                self.fixed_displacement = True
                self.traction = False

BC_settings("b").fixed_displacement

False

In [9]:
class edge_U:

    def __init__(self, edges, xy):

        if xy == "x":
            if edges[0] == "b":
                    self.BC = u_bottom
            if edges[0] == "t":
                    self.BC = u_top
            if edges[0] == "l":
                    self.BC = u_left
            if edges[0] == "r":
                    self.BC = u_right

        if xy == "y":
            if edges[0] == "b":
                    self.BC = v_bottom
            if edges[0] == "t":
                    self.BC = v_top
            if edges[0] == "l":
                    self.BC = v_left
            if edges[0] == "r":
                    self.BC = v_right

Now let's create a function that assigns these values the A and b matrix

In [10]:
def displacement_cell_BCs(A_matrix, b_matrix, k, edges, xy, U_old, U_old_old, U_previous):

    A_matrix[k,k] = boundaryCellDisplacement(edges, xy).a_P
    #an
    A_matrix[k,index(k).n] = - boundaryCellDisplacement(edges, xy).a_N        
    #as
    A_matrix[k,index(k).s] = - boundaryCellDisplacement(edges, xy).a_S   
    #ae
    A_matrix[k, index(k).e] = - boundaryCellDisplacement(edges, xy).a_E
    #aw
    A_matrix[k, index(k).w] = - boundaryCellDisplacement(edges, xy).a_W

    b_matrix[k] =(
                boundaryCellDisplacement.b_temp(U_old, U_old_old, k, xy)
                +
                boundaryCellDisplacement.b_diff(edges, k, xy, U_previous)
            )   

    return A_matrix, b_matrix

For the boundary points we assign the set values as the $b$ term and 1 as the $a_P$ term/

$$ [1][u] = [b]$$

$$ u = b$$

In [11]:
def displacement_point_BCs(A_matrix, b_matrix, k, edges, xy):
    
    A_matrix[k,k] = 1e10
    b_matrix[k] = edge_U(edges, xy).BC*1e10

    return A_matrix, b_matrix